In [1]:
import numpy as np
import scipy as sp
import pandas as pd


Spectral Clustering using Adjacency Matrix and Laplacian:
Adjacency:  optimizes on in group closeness
Laplacian: Average cut
A variant of the Laplacian: Normalized Cut

In general, Laplacian has better behaved/more stable convergenence properties of eigenvectors (makes sense when we are dealing with random graphs).  

TODO
make function for laplacian
make function for adjaceny matrix
make function for non backtracking matrix

make sampling method for blockmodel

make function for power method

make function for power method for subspace

In [37]:
#assume graphs are dictionaries of vertices with entries that are the incident vertices for now, we can change this later.  



#compute laplacian
    #f-Mf

        #averaging function
        
simple_graph = {
    0: [2,3],
    1: [2, 3],
    2: [3], 
    3: [0, 1]
}

simple_graph[1]

vector = np.array([1,1,1,1])

lst = [vector[i] for i in simple_graph[1]]


2

In [34]:
def AverageOperator(vertex, vector, graph):
    lst = [vector[i] for i in graph[vertex]]
    answer = np.sum(lst)
    return(answer)

In [36]:
AverageOperator(vertex = 2, vector = np.array([1,2,3,4]), graph = simple_graph)

4

In [51]:

def GraphLaplacian(graph, vector):
    #takes in graph and vector on graph indices, returns new vector
    new_vector = np.empty(len(vector),)
    for i in graph:
        #this step can be parallelized
        new_vector[i] = AverageOperator(vertex=i, vector = vector, graph = graph)
    return new_vector

In [52]:
GraphLaplacian(simple_graph, np.array([1,1,1,1]))

array([ 2.,  2.,  1.,  2.])

NameError: name 'install' is not defined